In [1]:
import requests
import json
from pprint import pprint
from config import apiKey

## Generate Meal Plan

In [4]:
## Generate Meal Plan
base_url = "https://api.spoonacular.com/recipes/mealplans/generate?"

timeFrame = input("Day or week? ")
targetCalories = input("How many calories? ")
diet = input("Diet (vegetarian, gluten free, etc. ) " )
exclude = input("Allergic to anything? ")

params = {
    "apiKey": apiKey,
    "timeFrame": timeFrame,
    "targetCalories": targetCalories,
    "diet": diet,
    "exclude": exclude
}
response = requests.get(base_url, params=params).json()

Day or week? day
How many calories? 2000
Diet (vegetarian, gluten free, etc. ) vegetarian
Allergic to anything? peanuts


In [5]:
pprint(response)

{'meals': [{'id': 663853,
            'image': 'Tropical-Breakfast-Quinoa-663853.jpg',
            'imageUrls': ['Tropical-Breakfast-Quinoa-663853.jpg'],
            'readyInMinutes': 45,
            'servings': 2,
            'title': 'Tropical Breakfast Quinoa'},
           {'id': 633754,
            'image': 'Baked-Ratatouille-633754.jpg',
            'imageUrls': ['Baked-Ratatouille-633754.jpg'],
            'readyInMinutes': 45,
            'servings': 1,
            'title': 'Baked Ratatouille'},
           {'id': 782619,
            'image': 'mushroom-goat-cheese-baked-eggs-782619.png',
            'imageUrls': ['mushroom-goat-cheese-baked-eggs-782619.png'],
            'readyInMinutes': 45,
            'servings': 2,
            'title': 'Mushroom Goat Cheese Baked Eggs'}],
 'nutrients': {'calories': 2019.55,
               'carbohydrates': 203.39,
               'fat': 106.25,
               'protein': 80.21}}


In [7]:
recipe_ids = [recipe['id'] for recipe in response['meals']]

In [8]:
recipe_ids

[663853, 633754, 782619]

In [34]:
recipe_url = f"https://api.spoonacular.com/recipes/informationBulk?includeNutrition=true?"
# \
# ids={recipe_ids[0]},{recipe_ids[1]},{recipe_ids[2]}?\
# apiKey={apiKey}"
params = {
    "ids": f"{recipe_ids[0]},{recipe_ids[1]},{recipe_ids[2]}",
    'apiKey': apiKey
}

recipe_response = requests.get(recipe_url, params=params).json()

In [35]:
pprint(recipe_response)

[{'aggregateLikes': 3,
  'analyzedInstructions': [{'name': '',
                            'steps': [{'equipment': [{'id': 404752,
                                                      'image': 'stock-pot.jpg',
                                                      'name': 'pot'}],
                                       'ingredients': [{'id': 20137,
                                                        'image': 'cooked-quinoa.png',
                                                        'name': 'cooked '
                                                                'quinoa'},
                                                       {'id': 19911,
                                                        'image': 'maple-syrup.png',
                                                        'name': 'maple syrup'},
                                                       {'id': 2010,
                                                        'image': 'cinnamon.jpg',
                                

                           'metaInformation': ['to taste'],
                           'name': 'salt and pepper',
                           'original': 'salt and pepper, to taste',
                           'originalName': 'salt and pepper, to taste',
                           'originalString': 'salt and pepper, to taste',
                           'unit': 'servings'},
                          {'aisle': 'Produce',
                           'amount': 4.0,
                           'consitency': 'solid',
                           'id': 11677,
                           'image': 'shallots.jpg',
                           'measures': {'metric': {'amount': 4.0,
                                                   'unitLong': 'larges',
                                                   'unitShort': 'large'},
                                        'us': {'amount': 4.0,
                                               'unitLong': 'larges',
                                               'u

In [46]:
pprint(recipe_response[0]['analyzedInstructions'])

[{'name': '',
  'steps': [{'equipment': [{'id': 404752,
                            'image': 'stock-pot.jpg',
                            'name': 'pot'}],
             'ingredients': [{'id': 20137,
                              'image': 'cooked-quinoa.png',
                              'name': 'cooked quinoa'},
                             {'id': 19911,
                              'image': 'maple-syrup.png',
                              'name': 'maple syrup'},
                             {'id': 2010,
                              'image': 'cinnamon.jpg',
                              'name': 'cinnamon'}],
             'length': {'number': 5, 'unit': 'minutes'},
             'number': 1,
             'step': 'In a medium size pot combine cooked quinoa, coconut '
                     'milk, crushed pineapple, cinnamon and maple syrup; stir '
                     'to combine. Bring to a boil and reduce heat. Cook for 5 '
                     'minutes, stirring occasionally.'},
      

In [79]:
## Recipe Equipment
# pprint(recipe_response[0]['analyzedInstructions'][0]['steps'][0]['equipment'][0]['name'])
equipment = recipe_response[0]['analyzedInstructions'][0]['steps'][0]['equipment']
if len(equipment) > 0:
    equipment = [equip['name'] for equip in equipment]
    


[{'id': 404752, 'name': 'pot', 'image': 'stock-pot.jpg'}]

In [78]:
## Recipe Steps
steps = recipe_response[0]['analyzedInstructions'][0]['steps']
recipe_steps = [step['step'] for step in steps]

'Top quinoa mixture with toasted coconut, pecans, sliced strawberries and balsamic vinegar.'